---

<h1 style="text-align: center;font-size: 40px;">TPS-July 2021: EDA with Baseline</h1>

---

<center><img src="https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05
"width="1000" height="600"></center>

---

> In this competition you have to predict the values of air pollution measurements over time, based on basic weather information (temperature and humidity) and the input values of 5 sensors.
The three target values that you have to predict are: target_carbon_monoxide, target_benzene, and target_nitrogen_oxides.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
plt.style.use("ggplot")
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv")
train.head()

In [1]:
train.info()

In [1]:
train.describe()

> <h4>Let's check, we have any Null values in our dataset or Not:</h4>

In [1]:
train.isnull().sum()

- So, we don't have any null values in our dataset

> Our date time column is a combination of year, month , day and hour. So we can make a new feature called **Hour** from our date time column and check if there is any relation between **Hour** and the other features like, is the amount of temperature, relative humidity, or target features increase or decrease in a certain period of time in a day. Then may be it'll be helpfull for making a good model.

In [1]:
year = train['date_time'].apply(lambda year: year.split("-")[0])
year.unique()

- This dataset contains value of 2 years 2010 & 2011

In [1]:
hour = train['date_time'].apply(lambda year: year.split(" ")[1].split(":")[0])
train["Hour"] = hour
train["Hour"] = train["Hour"].astype("int")
train["Hour"].unique()

> <h3>Let's now divide the hour into different time period of a day</h3>

In [1]:
def time_conv(hour):
    if hour in range(5,13):
        h = "Morning"
        return h
    elif hour in range(13,18):
        h = "Afternoon"
        return h
    elif hour in range(18,21):
        h = "Evening"
        return h
    elif hour in  [21,22,23,0,1,2,3,4]:
        h = "Night"
        return h

HR = train['Hour'].apply(time_conv)
train["Time"] = HR
HR.unique()

In [1]:
!pip install seaborn==0.10.0
import seaborn as sns
sns.__version__

In [1]:
import plotly.graph_objects as go
plt.figure(figsize=(16,6))
sns.lineplot(x = train["date_time"][:1000],
             y = train["relative_humidity"][:1000],hue = "Time",data = train
             )
plt.xticks([""])
plt.show()

In [1]:
fig,ax = plt.subplots(figsize=(16,6))
sns.lineplot(x = train["date_time"][:1000],
             y = train["deg_C"][:1000],hue = "Time",data = train
             )
plt.xticks([""])
plt.show()

> Observations:
- The temperature stays low at Night
- Goes High at Afternoon and Evening

> <h3>Relation between deg_c and the 3 target classes</h3>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["deg_C"], y = train[y],hue = train["Time"])

plt.suptitle("Relation between deg_c and the 3 target classes",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

> Observation:
- Seems like there is no Relation between deg_c and the 3 target Variables

> <h4>Sensor_1's relation with 3 target variables:</h4>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["sensor_1"], y = train[y],hue = train["Time"])

plt.suptitle("Sensor_1's relation with 3 target variables",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

> Observation:
- Sensor_1 has a positive relationship with all 3 target variables
- As Sensor_1 increases the amount of all 3 target variables also increases

> <h4>Sensor_2's relation with 3 target variables:</h4>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["sensor_2"], y = train[y],hue = train["Time"])

plt.suptitle("Sensor_2's relation with 3 target variables",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

> Observation:
- Seems like Sensor 2 is also highly correlated with all 3 target variables

> <h4>Sensor 3's relation with 3 target variables:</h4>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["sensor_3"], y = train[y],hue = train["Time"])

plt.suptitle("Sensor_3's relation with 3 target variables",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

> Observation:
- Sensor 3 is negatively correlated with all 3 target variables
- As Sensor 3 increases the amount of all 3 target variables decreases   

> <h4>Sensor 4's relation with 3 target variables:</h4>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["sensor_4"], y = train[y],hue = train["Time"])

plt.suptitle("Sensor_4's relation with 3 target variables",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

> Observation:
- Looks like Sensor 4 is highly correlated with benzene and carbon monoxide, as it increases benzene and carbon monoxide also increases. 
- But it not much correlated with nitrogen oxides

> <h4>Sensor 5's relation with 3 target variables:</h4>

In [1]:
plt.figure(figsize=(16,6))
target_label = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
for i in range(3):
    plt.subplot(1,3,i+1)
    y = target_label[i]
    sns.scatterplot(data = train,x=train["sensor_5"], y = train[y],hue = train["Time"])

plt.suptitle("Sensor 5's relation with 3 target variables",x=0.5,y = 1.04)    
plt.tight_layout()
plt.show()

>Observation:
- Looks like Sensor 5 is also positively correlated with all 3 target variables

> <h4>Correlation between Features:</h4>

In [1]:
corr_mat = train.corr()
mask = np.array(corr_mat)
mask[np.tril_indices_from(mask)] = False
fig = plt.gcf()
fig.set_size_inches(16,9)
sns.heatmap(corr_mat,mask = mask,square = True,annot= True,cbar = True)
plt.show()

> <h4>Distribution of features:</h4>

In [1]:
plt.figure(figsize=(16,8))
features = train.columns[1:9]
for i,col in enumerate(features):
    plt.subplot(2,4,i+1)
    sns.distplot(train[col])
plt.suptitle("Distribution of Features",x=0.5,y=1.03)
plt.tight_layout()
plt.show()

> <h4>Distribution of Target Classes:</h4>

In [1]:
plt.figure(figsize=(16,5))
for i,col in enumerate(target_label):
    plt.subplot(1,3,i+1)
    sns.distplot(train[col])
plt.suptitle("Distribution of Target Classes",x=0.5,y=1.03)
plt.tight_layout()
plt.show()

> <h4>Is there any relation between Time and 3 target variables?</h4>

In [1]:
fig,ax = plt.subplots(figsize=(16,6))
sns.lineplot(x = train["date_time"][:1000],
             y = train["target_carbon_monoxide"][:1000],hue = "Time",data = train
             )
plt.xticks([""])
plt.show()

>Observation:
- Seems like **Carbon Monoxide** stays high at Morning and Evening
- Stays low at Night and Afternoon most of the time

In [1]:
fig,ax = plt.subplots(figsize=(16,6))
sns.lineplot(x = train["date_time"][:1000],
             y = train["target_benzene"][:1000],hue = "Time",data = train
             )
plt.xticks([""])
plt.show()

>Observation:
- Seems like, amount of  **Benzene** also stays high at Morning and Evening
- Stays low at Night and Afternoon most of the time

In [1]:
fig,ax = plt.subplots(figsize=(16,6))
sns.lineplot(x = train["date_time"][:1000],
             y = train["target_nitrogen_oxides"][:1000],hue = "Time",data = train
             )
plt.xticks([""])
plt.show()

>Observation:
- Seems like **Nitrogen Oxides** stays high at Morning most of the time
- Stays low at Night, Evening and Afternoon most of the time

> <h4>Simple XGBRegressor Baseline:</h4>

In [1]:
import warnings
warnings.filterwarnings('ignore')
train_col = train.columns[1:9]
x = train[train_col]
x["Time"] = train["Time"]
target_col = train.columns[9:12]
y = train[target_col]

In [1]:
x = pd.get_dummies(x,drop_first = True)
x.head()

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor

In [1]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

> Separating hour from test dataset:

In [1]:
hour = test['date_time'].apply(lambda year: year.split(" ")[1].split(":")[0])
test["Hour"] = hour
test.head(2)

In [1]:
test["Hour"] = test["Hour"].astype("int")
HR_T = test['Hour'].apply(time_conv)
test["Time"] = HR_T
HR_T.unique()

In [1]:
test.drop(["Hour","date_time"],axis=1,inplace=True)
test = pd.get_dummies(test,drop_first = True)
test.head(2)

In [1]:
xgb = XGBRegressor()

clf=xgb.fit(x_train,y_train["target_carbon_monoxide"])
sub["target_carbon_monoxide"] = clf.predict(test)

clf=xgb.fit(x_train,y_train["target_benzene"])
sub["target_benzene"] = clf.predict(test)

clf=xgb.fit(x_train,y_train["target_nitrogen_oxides"])
sub["target_nitrogen_oxides"] = clf.predict(test)

In [1]:
sub.head(2)

In [1]:
sub.to_csv("Submision.csv", index = False)